In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
X = df[['koi_fpflag_nt', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration']]
# y is koi_disposition
y = df["koi_disposition"]
target_names = ["CONFIRMED", "FALSE POSITIVE","CANDIDATE"]
# Drop koi_disposition column
#X = df.drop("koi_disposition", axis=1)
X.head()


,koi_fpflag_nt,koi_period,koi_time0bk,koi_impact,koi_duration
0,0,54.418383,162.513840,0.586,4.50700
1,0,19.899140,175.850252,0.969,1.78220
2,0,1.736952,170.307565,1.276,2.40641
3,0,2.525592,171.595550,0.701,1.65450
4,0,4.134435,172.979370,0.762,3.14020


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
X_train.head()

,koi_fpflag_nt,koi_period,koi_time0bk,koi_impact,koi_duration
6122,0,6.768901,133.077240,0.150,3.61600
6370,0,0.733726,132.020050,0.291,2.30900
2879,1,7.652707,134.460380,0.970,79.89690
107,0,7.953547,174.662240,0.300,2.63120
29,0,4.959319,172.258529,0.831,2.22739


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [9]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.40663742132366965
Testing Data Score: 0.4193363844393593


In [10]:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

SVC(gamma='auto', kernel='linear')

In [11]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.576


In [12]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

                precision    recall  f1-score   support

     CONFIRMED       0.69      0.20      0.31       411
FALSE POSITIVE       0.43      0.55      0.48       484
     CANDIDATE       0.66      0.77      0.71       853

      accuracy                           0.58      1748
     macro avg       0.59      0.51      0.50      1748
  weighted avg       0.60      0.58      0.55      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
# Create the GridSearchCV model
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

SVC(decision_function_shape=None, gamma='auto', kernel='linear')

In [14]:
# Train the model with GridSearch
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5],
              'gamma': [0.0001, 0.0005]}
grid = GridSearchCV(model, param_grid, verbose=3)

# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ..............................C=1, gamma=0.0001; total time=  18.9s
[CV 2/5] END ..............................C=1, gamma=0.0001; total time=  16.9s
[CV 3/5] END ..............................C=1, gamma=0.0001; total time=  20.4s
[CV 4/5] END ..............................C=1, gamma=0.0001; total time=  16.8s
[CV 5/5] END ..............................C=1, gamma=0.0001; total time=  17.0s
[CV 1/5] END ..............................C=1, gamma=0.0005; total time=  19.0s
[CV 2/5] END ..............................C=1, gamma=0.0005; total time=  17.0s
[CV 3/5] END ..............................C=1, gamma=0.0005; total time=  20.5s
[CV 4/5] END ..............................C=1, gamma=0.0005; total time=  16.9s
[CV 5/5] END ..............................C=1, gamma=0.0005; total time=  17.1s
[CV 1/5] END ..............................C=5, gamma=0.0001; total time=  55.6s
[CV 2/5] END ..............................C=5, g

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5], 'gamma': [0.0001, 0.0005]}, verbose=3)

In [16]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 5, 'gamma': 0.0001}
0.5803966336532794


# Save the Model

In [17]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'svm.sav'
joblib.dump(model, filename)

['svm.sav']